In [7]:
import pandas as pd
import os as os
import mendeleev as ptable
import numpy as np
from Determine_Bonds_Get_Input import find_orbitals_get_characteristics, build_graph_tensor

In [43]:
class g16_input_file:
    def __init__(self, seed_structure_directory, molecular_formula, job_type, input_directory, output_directory, AO_basis, polarization, diffuse, job_title, SCF_optimization, csv_filepath, record_filepath, record_filename, element, n_CPU, allowed_memory):
        self.seed_structure_directory = seed_structure_directory
        self.molecular_formula = molecular_formula
        self.job_type = job_type
        self.input_directory = input_directory
        self.output_directory = output_directory
        self.AO_basis = AO_basis
        self.polarization = polarization
        self.job_title = job_title
        self.SCF_optimization = SCF_optimization
        self.csv_filepath = csv_filepath
        self.record_filepath = record_filepath
        self.record_filename = record_filename
        self.element = element
        self.n_CPU = n_CPU
        self.allowed_memory = allowed_memory

    def generate_input_file(self, molecular_formula, job_type, input_directory, output_directory, seed_structure_directory, csv_filepath, record_filepath, record_filename, n_CPU, allowed_memory):
        input_filepath = os.path.join(input_directory, str(molecular_formula))
        input_file = open(input_filepath, "w")
        seed_structure = seed_structure_directory + "/" + str(molecular_formula) + ".csv"
        energy_checkpoint_path = os.path.join(output_directory, str(molecular_formula))
        geom_checkpoint_path = os.path.join(output_directory, str(molecular_formula) + "_geom")
        energy_chk = open(energy_checkpoint_path, "w")
        geom_chk = open(geom_checkpoint_path, "w")
        energy_chk_name = str(molecular_formula) + ".chk"
        geom_chk_name = str(molecular_formula) + "_geom" + ".chk"
        if job_type == "ground_state_energy_calculation":
            job_abbrev = "SP"
            input_file.write("%chk=" + energy_chk_name)
            input_file.write("\n")
        elif job_type == "geometry_optimization":
            job_abbrev = "Opt"
            input_file.write("%chk=" + geom_chk_name)
            input_file.write("\n")
        graph_tensor = build_graph_tensor(csv_filepath, record_filepath, record_filename)
        molecular_characteristics = find_orbitals_get_characteristics(molecular_formula = molecular_formula, seed_structure_directory = seed_structure_directory, element = None, graph_tensor = graph_tensor)
        symbol, Z, block = molecular_characteristics.get_symbols_and_numbers(molecular_formula = molecular_characteristics.molecular_formula, seed_structure_directory = molecular_characteristics.seed_structure_directory, graph_tensor = molecular_characteristics.graph_tensor)
        if "d" in block or "f" in block:
            AO_basis = "triple_zeta"
        elif "C" in symbol:
            AO_basis = "6-31G"
        elif "Na" in symbol or "K" in symbol or "Mg" in symbol or "Ca" in symbol or "Li" in symbol or "Be" in symbol:
            AO_basis = "LANL_double_zeta"
            # remember: input LANL2DZ for this case
        else:
            AO_basis = "STO-3G"
        input_file.write("%nprocs=" + str(n_CPU))
        input_file.write("\n")
        input_file.write("%mem=" + allowed_memory)
        input_file.write("\n")
        if AO_basis == "triple_zeta":
            
        return AO_basis
        
        
        

In [44]:
def find_atomic_symbol(element):
    characteristic_list = []
    blanks = np.array([])
    for i in np.arange(0, len(str(element))):
        characteristic_list.append(str(element)[i])
    for j in np.arange(0, len(characteristic_list)):
        if characteristic_list[j] == " ":
            blanks = np.append(blanks, j)
    atomic_symbol = str(element)[int(blanks[0]):int(blanks[1])]
    return atomic_symbol

atomic_numbers = np.array([])
element_set = ptable.get_all_elements()
for i in np.arange(0, len(element_set)):
    if find_atomic_symbol(element_set[i]) == find_atomic_symbol(element_set[56]):
        print('lanthanum')
        print(element_set[i].block)

lanthanum
d


In [46]:
csv_filepath = "/users/haydenprescott/documents/test.csv"
record_filepath = "/users/haydenprescott/test_data.tfrecord" 
record_filename = "test_data.tfrecord"

gaussian_input = g16_input_file(seed_structure_directory = "/users/haydenprescott/documents", molecular_formula = "La3O4", job_type = "ground_state_energy_calculation", input_directory = "/users/haydenprescott/documents", output_directory = "/users/haydenprescott/documents/gaussian_test_data", AO_basis = " ", polarization = None, diffuse = None, job_title = "seed structure ground state", SCF_optimization = 'tight', csv_filepath = csv_filepath, record_filepath = record_filepath, record_filename = record_filename, element = None, n_CPU = 2, allowed_memory = "1000MB")
print(gaussian_input.generate_input_file(molecular_formula = gaussian_input.molecular_formula, job_type = gaussian_input.job_type, input_directory = gaussian_input.input_directory, output_directory = gaussian_input.output_directory, seed_structure_directory = gaussian_input.seed_structure_directory, csv_filepath = gaussian_input.csv_filepath, record_filepath = gaussian_input.record_filepath, record_filename = gaussian_input.record_filename))
                                         

triple_zeta


In [30]:
graph_tensor = build_graph_tensor(csv_filepath, record_filepath, record_filename)
print(molecular_characteristics.get_symbols_and_numbers(molecular_formula = molecular_characteristics.molecular_formula, seed_structure_directory = molecular_characteristics.seed_structure_directory, graph_tensor = molecular_characteristics.graph_tensor))                                                     

(array(['La', 'O', 'O', 'La', 'O', 'O'], dtype=object), array([57.,  8.,  8., 57.,  8.,  8.]), array(['d', 'p', 'p', 'd', 'p', 'p'], dtype='<U32'))
